# Language Model Practical 7

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
!pip install keras.preprocessing

In [16]:
from tensorflow import keras
import numpy
from numpy import array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

data = """ Jack and Jill went up the hill .\n To fetch a pail of water .\n Jack fell down and broke his crown .\n And Jill came tumbling after ."""
print("Data:", data, type(data))

data_splitted=data.split('\n') #returns a list of strings
print("Data_Splitted:", data_splitted, type(data_splitted))

tokenizer=Tokenizer(filters='!"#$%&()*+,-/:;<=>?@[\]^_`{|}~')

tokenizer.fit_on_texts(data_splitted) #learns a vocabulary
print("Word Indices:", tokenizer.word_index) #tokenizer.word_index is a dictionary

vocab_size = len(tokenizer.word_index) + 1
print("Vocab Size:", vocab_size)

sequences=tokenizer.texts_to_sequences(data_splitted) #list of list
l=len(sequences)
print("Sequences:",sequences, type(sequences), l)

X=list()
y=list()

for i in range(len(sequences)):
    X.insert(i,sequences[i][:-1])
    y.insert(i,sequences[i])

print("X=", X, "y=",y, type(X), type(y))

maxlen = max([len(sequence) for sequence in X])
print("Maxlen:",maxlen)

#X=array(X)
#y=array(y)
#X=numpy.reshape(X,newshape=(l,-1))
#y=numpy.reshape(y,newshape=(l,-1))

X=pad_sequences(X,maxlen=maxlen+1,padding='pre') # +1 to have 0 as the first input
print("X:",X, type(X), X.shape)
y=pad_sequences(y,maxlen=maxlen+1,padding='pre')
print("y:",y, type(y), y.shape)

#z=numpy.
#for i in range(l):
#  z[i]=to_categorical(y[i],num_classes=vocab_size)

#print("X=", X, "y=",y)
y=to_categorical(y,num_classes=vocab_size)

print("X=", X, "y=",y, type(X), type(y), X.shape, y.shape)
model=Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=10))
model.add(SimpleRNN(units=50, return_sequences=True))
model.add(Dense(units=vocab_size,activation='softmax'))
model.summary()

model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=['accuracy'])

model.fit(X, y, epochs=200, verbose=0)

#sample_seq_wo_seed(model, tokenizer, 10, 23)



def prob_of_input_sentence(model, tokenizer, sentence):
    print("Input Sentence:", sentence)
    encoded=tokenizer.texts_to_sequences([sentence])[0]
    encoded.insert(0,0)
    encoded=array(encoded)
    encoded=numpy.reshape(encoded,newshape=(1,-1))
    print("Encoded:", encoded, encoded.shape)
    prob=model.predict(encoded, verbose=0)
    print("Prob:", prob, prob.shape)
    probability=1
    for i in range(prob.shape[1]-1):
        probability = probability * prob[0,i,encoded[0,i+1]]
    print("Probability of Sentence", "\"", sentence, "\"", "is:", probability)

print("-------------------Probability of Input Sentence------------------------------")
prob_of_input_sentence(model, tokenizer, "Jack and Jill Went")
prob_of_input_sentence(model, tokenizer, "and Jill Went up")
prob_of_input_sentence(model, tokenizer, "went up the hill")
prob_of_input_sentence(model, tokenizer, "jack and Jill Went up the hill .")
prob_of_input_sentence(model, tokenizer, "to fetch a pail")
prob_of_input_sentence(model, tokenizer, "fetch a pail")
prob_of_input_sentence(model, tokenizer, "to fetch a pail of water")
prob_of_input_sentence(model, tokenizer, "jack fell down and")
prob_of_input_sentence(model, tokenizer, "jack fell down and broke")
prob_of_input_sentence(model, tokenizer, "fell down and broke")
prob_of_input_sentence(model, tokenizer, "jack fell down and broke his crown")
prob_of_input_sentence(model, tokenizer, "and jill came tumbling")
prob_of_input_sentence(model, tokenizer, "g after")
prob_of_input_sentence(model, tokenizer, "and jill came tumbling after .")
print("-------------------------------------------------------------------------------")

prob_of_input_sentence(model, tokenizer, "and jill came walkings after .")

from tensorflow import keras
from keras.layers import Dense, Input
from keras.models import Model, Sequential
from keras.datasets import mnist
import numpy as np
from tensorflow.keras.utils import to_categorical
#from tensorflow.keras.utils import to_categorical


(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = np.reshape(x_train, newshape=(60000, 784)).astype('float32')
x_test = np.reshape(x_test, newshape=(10000, 784)).astype('float32')
y_train=to_categorical(y_train,num_classes=10)
y_test=to_categorical(y_test,num_classes=10)
x_train=x_train/255
x_test=x_test/255



input_main=Input(shape=(784,))
h1=Dense(units=100, activation='sigmoid')(input_main)
o1=Dense(units=784, activation='sigmoid')(h1)
autoencoder1=Model(inputs=input_main, outputs=o1)
autoencoder1.summary()
autoencoder1.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

print(type(autoencoder1.layers[0].get_weights()))
print(type(autoencoder1.layers[1].get_weights()))
print(type(autoencoder1.layers[2].get_weights()))

print(len(autoencoder1.layers[0].get_weights()))
print(len(autoencoder1.layers[1].get_weights()))
print(len(autoencoder1.layers[2].get_weights()))

print(type(autoencoder1.layers[1].get_weights()[0]))
print(type(autoencoder1.layers[1].get_weights()[1]))
print(autoencoder1.layers[1].get_weights()[0].shape)
print(autoencoder1.layers[1].get_weights()[1].shape)

print(type(autoencoder1.layers[2].get_weights()[0]))
print(type(autoencoder1.layers[2].get_weights()[1]))
print(autoencoder1.layers[2].get_weights()[0].shape)
print(autoencoder1.layers[2].get_weights()[1].shape)

print(autoencoder1.layers[1].get_weights()[0][50,50])
print(autoencoder1.layers[1].get_weights()[1][10])
print(autoencoder1.layers[2].get_weights()[0][50,50])
print(autoencoder1.layers[2].get_weights()[1][10])

print("Training Autoencoder1:")
autoencoder1.fit(x_train,x_train,epochs=5)

print(autoencoder1.layers[1].get_weights()[0][50,50])
print(autoencoder1.layers[1].get_weights()[1][10])
print(autoencoder1.layers[2].get_weights()[0][50,50])
print(autoencoder1.layers[2].get_weights()[1][10])

autoencoder1_hidden_output=autoencoder1.layers[1].output
trimmed_autoencoder1=Model(inputs=input_main, outputs=autoencoder1_hidden_output)
x_train_ae2=trimmed_autoencoder1.predict(x_train)
x_test_ae2=trimmed_autoencoder1.predict(x_test)
print(x_train_ae2.shape)


inputs_ae2=Input(shape=(100,))
h2=Dense(units=50, activation='sigmoid')(inputs_ae2)
o2=Dense(units=100, activation='sigmoid')(h2)
autoencoder2=Model(inputs=inputs_ae2, outputs=o2)
autoencoder2.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
print(autoencoder2.layers[1].get_weights()[0].shape)
print(autoencoder2.layers[2].get_weights()[0].shape)
print("Training Autoencoder2:")
autoencoder2.fit(x_train_ae2, x_train_ae2, epochs=5)

autoencoder2_hidden_output=autoencoder2.layers[1].output
trimmed_autoencoder2=Model(inputs=inputs_ae2, outputs=autoencoder2_hidden_output)
x_train_clf=trimmed_autoencoder2.predict(x_train_ae2)
x_test_clf=trimmed_autoencoder2.predict(x_test_ae2)
print(x_train_clf.shape)

inputs_clf=Input(shape=(50,))
f_output=Dense(units=10, activation='softmax')(inputs_clf)
clf=Model(inputs=inputs_clf, outputs=f_output)
clf.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(clf.layers[1].get_weights()[0].shape)
print("Training Classifier:")
clf.fit(x_train_clf, y_train, epochs=5)


print(clf.evaluate(x_test_clf,y_test))


new_model=Sequential()
new_model.add(autoencoder1.layers[0])
new_model.add(autoencoder1.layers[1])
new_model.add(autoencoder2.layers[1])
new_model.add(clf.layers[-1])
new_model.summary()
new_model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
print(new_model.layers[1].get_weights()[0].shape)
print(autoencoder2.layers[1].get_weights()[0].shape)
print("Fine tuning:")
new_model.fit(x_train, y_train, epochs=5)




Data:  Jack and Jill went up the hill .
 To fetch a pail of water .
 Jack fell down and broke his crown .
 And Jill came tumbling after . <class 'str'>
Data_Splitted: [' Jack and Jill went up the hill .', ' To fetch a pail of water .', ' Jack fell down and broke his crown .', ' And Jill came tumbling after .'] <class 'list'>
Word Indices: {'.': 1, 'and': 2, 'jack': 3, 'jill': 4, 'went': 5, 'up': 6, 'the': 7, 'hill': 8, 'to': 9, 'fetch': 10, 'a': 11, 'pail': 12, 'of': 13, 'water': 14, 'fell': 15, 'down': 16, 'broke': 17, 'his': 18, 'crown': 19, 'came': 20, 'tumbling': 21, 'after': 22}
Vocab Size: 23
Sequences: [[3, 2, 4, 5, 6, 7, 8, 1], [9, 10, 11, 12, 13, 14, 1], [3, 15, 16, 2, 17, 18, 19, 1], [2, 4, 20, 21, 22, 1]] <class 'list'> 4
X= [[3, 2, 4, 5, 6, 7, 8], [9, 10, 11, 12, 13, 14], [3, 15, 16, 2, 17, 18, 19], [2, 4, 20, 21, 22]] y= [[3, 2, 4, 5, 6, 7, 8, 1], [9, 10, 11, 12, 13, 14, 1], [3, 15, 16, 2, 17, 18, 19, 1], [2, 4, 20, 21, 22, 1]] <class 'list'> <class 'list'>
Maxlen: 7
X: [[

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 100)            │        78,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 10)             │           510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 84,060 (328.36 KB)

 Trainable params: 84,060 (328.36 KB)

 Non-trainable params: 0 (0.00 B)

(100, 50)
(100, 50)
Fine tuning:
Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9050 - loss: 0.3269
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9519 - loss: 0.1706
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9647 - loss: 0.1217
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9736 - loss: 0.0951
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9791 - loss: 0.0736


# Review

The model is trained on a small dataset consisting of sentences split into sequences of words. The code preprocesses the data, tokenizes the words, and converts them into sequences. Then, it builds an RNN model with an embedding layer followed by a SimpleRNN layer and a Dense layer. The model is trained to predict the next word in a sequence given the previous words. Finally, the trained model is used to calculate the probability of input sentences.

Deep Autoencoder and Classifier Integration: This part involves building a deep autoencoder followed by a classifier. The autoencoder is designed to compress the input data into a lower-dimensional representation, and the classifier is trained to classify the compressed representations into different classes. The code loads the MNIST dataset, preprocesses it, and constructs two autoencoder models: autoencoder1 and autoencoder2. These autoencoders are then used to extract features, and a classifier is trained on these features to classify the MNIST digits.

Additionally, the code combines the two parts by fine-tuning the integrated model using both the autoencoder and the classifier. The final model is trained on the MNIST dataset to perform classification after feature extraction using the autoencoders.